In [21]:
#manipulate dataframes in python
import pandas as pd
import sys,os

#make API calls with python
import requests

# sys.path.insert(0, '/path/to/application/app/folder') used to import local python file, in this case local keys
sys.path.insert(0, '../src/')
import localvars

#allows us to store results of API call cleanly
import json

# List of Years
strYears =["2013", "2014", "2015", "2016", "2017"]

# Update Variables for the Correct State and County
# TN = 47, Montgomery County, TN = 125
intState = 47
intCounty = 125

tables = pd.read_csv("../src/data/tables.csv").to_dict(orient="row")
tablesdf = pd.DataFrame(tables)
tablesdf2 = tablesdf.set_index("table", drop = False)
for index, row in tablesdf2.iterrows():
    tablename= row['table']
    #print("value: " + tablesdf2.loc[tablename,'file'])
    # Using for loop 
    metric = pd.DataFrame() #creates a new dataframe that's empty
    for i in strYears: 

        #construct the API call we will use
        #the census calls query for block group; api calls can be changed for tract or other geography
        #see: https://api.census.gov/data/2017/acs/acs5.html and geography and example links
        if tablename[:1]=="B":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=block%%20group:*&in=state:%i&in=county:%i&in=tract:*&key=%s"
        elif tablename[:1]=="C":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5?get=%s&for=block%%20group:*&in=state:%i&in=county:%i&in=tract:*&key=%s"   
        elif tablename[:1]=="D":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/profile?get=%s&for=block%%20group:*&in=state:%i&in=county:%i&in=tract:*&key=%s" 
        elif tablename[:1]=="S":
            baseAPI = "https://api.census.gov/data/%s/acs/acs5/subject?get=%s&for=block%%20group:*&in=state:%i&in=county:%i&in=tract:*&key=%s"

        calledAPI = (baseAPI % (i, tablename, intState, intCounty, localvars.apiKey))

        #call the API and collect the response
        
        try:
            response = requests.get(calledAPI,timeout=60)
        except json.decoder.JSONDecodeError:
            print("json.decoder.JSONDecodeError: Waiting 5 Seconds")
            time.sleep(5)
            response = requests.get(calledAPI,timeout=60)
        

        #load the response into a JSON, ignoring the first element which is just field labels
        try:
            formattedResponse = json.loads(response.text)[1:]
        except json.decoder.JSONDecodeError:
            # Error occurs when table does not exist for a given year (internet and insurance was not collected in 2013). Adds 0 values 
            # for these years, uses a table with no data for Census Blocks. blanks.json works for Montgomery County, TN
            with open("../src/data/blank.json", "r") as read_file:
                formattedResponse = json.load(read_file)[1:]
            pass
        except:
            print(calledAPI)
            print("Unexpected error:", sys.exc_info()[0])
            raise

        #flip the order of the response from [population, zipcode] -> [zipcode, population]
        formattedResponse = [item[::-1] for item in formattedResponse]

        #store the response in a dataframe
        strColYear = "y_" + i

        metrictemp = pd.DataFrame(columns=['id', '2', '3', '4', strColYear], data=formattedResponse)
        metrictemp["id"] = metrictemp["2"]  + "-" + metrictemp["id"]
        for index1, row1 in metrictemp.iterrows():
            if row1[strColYear] is None:
                row1[strColYear] = "0"
            if float(row1[strColYear])<0:
                #row[strColYear]="0"
                row1[strColYear] = "0"
        del metrictemp['2']
        del metrictemp['3']
        del metrictemp['4']
        if i == "2013":
            metric=metrictemp
        else: 
            metric = metric.merge(metrictemp, on='id', how='outer')
    

    #save that dataframe to a CSV spreadsheet
    strFileName=row['file']
    strFile = "../src/data/ProcessedData/" + strFileName + ".csv"
    strJson ="../src/data/json/m" + strFileName[1:]+ ".json"
    metric.to_csv(strFile, index=False)
    print("Output: " + strFile)




Output: ../src/data/ProcessedData/r1.csv
Output: ../src/data/ProcessedData/r2.csv
Output: ../src/data/ProcessedData/r3.csv
Output: ../src/data/ProcessedData/r4.csv
Output: ../src/data/ProcessedData/r5.csv
Output: ../src/data/ProcessedData/r6.csv
Output: ../src/data/ProcessedData/r7.csv
Output: ../src/data/ProcessedData/r8.csv
Output: ../src/data/ProcessedData/r9.csv
Output: ../src/data/ProcessedData/r10.csv
Output: ../src/data/ProcessedData/r11.csv
Output: ../src/data/ProcessedData/r12.csv
Output: ../src/data/ProcessedData/r13.csv
Output: ../src/data/ProcessedData/r14.csv
Output: ../src/data/ProcessedData/r15.csv
Output: ../src/data/ProcessedData/r16.csv
Output: ../src/data/ProcessedData/r17.csv
Output: ../src/data/ProcessedData/r18.csv
Output: ../src/data/ProcessedData/r19.csv
Output: ../src/data/ProcessedData/r20.csv
Output: ../src/data/ProcessedData/r21.csv
Output: ../src/data/ProcessedData/r22.csv
Output: ../src/data/ProcessedData/r23.csv
Output: ../src/data/ProcessedData/r24.csv
O